In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import cvxpy as cvx

In [2]:
def get_year(df_folsom, year, h, year2=None):
    # This function grabs the specified single or multiyear 
    # data from df_folsom dataframe        
    if (year2==None):
        year2=year
    start = "%s-10-01" % (year-1)
    if (h==0):
        end = "%s-09-30" % (year2)
    else:
        end = "%s-10-%s" % (year2, f'{h:02}') 
    df_Q = df_folsom[start:end]
    return df_Q

In [3]:
def get_obj(df):
    # Calculate the objective value with an universal equation    
    # from the optimized release df.u 
    # storm benchmark set as 40 TAF.     
    WS = sum(((5 - df.u.values).clip(0,5))**2)*100 - df.x.values[-1]
    FC = sum(((df.u.values-40).clip(0,1000))**2)
    objective = WS+FC
    return objective

In [4]:
def folsom_complete(df_folsom, year1, umax, year2=None, detail='n', solver=''):
    # run one-shot optimization over the whole time frame
    # hence the perfect foresight model
    
    # set up parameters
    df_Q = get_year(df_folsom, year1, 0, year2)
    unit_conv = 60*60*24/43560/1000
    Q = df_Q.inflow.values.astype(int)*unit_conv
    T = len(Q)
    K = 975 # reservoir capacity
    d = 5*np.ones(T) # target demand (TAF/daily)

    # need some empty arrays to fill in
    x_save = np.zeros(T+1)
    u_save = np.zeros(T)
    shortage_save = np.zeros(T)
    spill_save = np.zeros(T)
    collective_obj = 0

    x = cvx.Variable(T+1)
    u = cvx.Variable(T)
    WS = cvx.sum((cvx.pos(d-u)*10)**2) - 0.01*x[-1] #0.05 arbitrary too powerful. should not induce any shortage in wet years
    FC = cvx.sum(cvx.pos(u-umax)**2)

    # objective function
    obj = cvx.Minimize(FC+WS)    

    # constraints (define separately, then concatenate the lists)
    c_mass_balance = [x[1:] == x[:-1] - u + Q] # state transition
    c_release = [u >= 0] # release lower/upper bounds 
    c_storage = [x >= 0, x <= K] # storage lower/upper bounds
    c_init_final = [x[0] == df_Q.storage.values[0]/1000, x[-1] >= 0]
    constraints = c_mass_balance + c_release + c_storage + c_init_final

    # solve for optimization problem with two different solvers
    prob = cvx.Problem(obj, constraints)
    if (solver == 'ECOS'):
      prob.solve(solver='ECOS')
    else:
      prob.solve()  

    # save in big array
    collective_obj += obj.value
    x_save = x.value.flatten()
    u_save = u.value.flatten()
    shortage_save = (d-u.value.flatten()).clip(0,999)
    spill_save = (u.value.flatten()-d).clip(0,999)
        
    # add these results into the original dataframe
    df_Q.inflow = Q
    df_Q['x'] = pd.Series(x_save[:-1], index=df_Q.index)
    df_Q['u'] = pd.Series(u_save, index=df_Q.index)
    df_Q['shortage'] = pd.Series(shortage_save, index=df_Q.index)
    df_Q['spill'] = pd.Series(spill_save, index=df_Q.index)
        
    return collective_obj, df_Q  

In [5]:
def visualize(df_Q):
    # visualize a single optimization result (single year or multiyear)  

    # set figure size
    plt.rcParams['figure.figsize'] = [10, 13] 

    # plot the state variable (storage) and control variable (release)
    plt.subplot(4,1,1)
    plt.plot(df_Q.inflow)
    plt.ylabel('inflow (TAF)')
    
    plt.subplot(4,1,2)
    plt.plot(df_Q.x)
    plt.ylim([0,1000])
    plt.axhline(975, linestyle=':')
    plt.ylabel('Storage (TAF)')

    plt.subplot(4,1,4)
    plt.plot(df_Q.shortage.round(decimals=2), color='seagreen')
#    plt.ylim([0,5])
    plt.ylabel('Shortage (TAFD)')
    plt.xlabel('Days (from 1/1/1997)')
    
    plt.subplot(4,1,3)
    plt.plot(df_Q.spill)
    plt.axhline(40, linestyle=':')      # 40/D is normal operation 
    plt.axhline(230, linestyle=':')     # 230/D is the spillway channel maximum capacity
    plt.ylabel('Spill (TAF/D)')
    plt.show()

    plt.rcParams['figure.figsize'] = [10, 5]
    
    # any rules that can be referred?
    df = df_Q.resample('M').agg({'x': 'first', 'inflow': np.sum, 'u': np.sum})
    x_plot = df.x + df.inflow
    plt.scatter(x_plot.values, df.u.values)
    plt.plot([0,1000],[0,1000], color='k', linewidth=2)
    plt.plot([975, 975+1000],[0,1000], color='k', linewidth=2)
    plt.plot([0,2000],[150,150], '--', color='k')
    plt.ylim([0,1000])
    plt.xlim([0,2000])
    plt.xlabel('Water Available (S+Q)')
    plt.ylabel('Release (u)')
    plt.show()

In [6]:
def folsom_limit(df_folsom, year1, h, umax, year2=None, detail='n', solver=''):
    # run daily optimization over the whole horizon h days
    # hence the limited foresight model

    # set up parameters
    df_Q = get_year(df_folsom, year1, h, year2)
    unit_conv = 60*60*24/43560/1000
    Q = df_Q.inflow.values.astype(int)*unit_conv
    T = len(Q)
    K = 975 # reservoir capacity
    d = 5*np.ones(h) # target demand (TAF/daily)

    # need some empty arrays to fill in
    x_save = np.zeros(T-h+1)
    u_save = np.zeros(T-h)
    shortage_save = np.zeros(T-h)
    spill_save = np.zeros(T-h)
    collective_obj = 0

    # loop daily for solution
    for i in range(0,T-h):
        x = cvx.Variable(h+1)
        u = cvx.Variable(h)
        WS = cvx.sum((cvx.pos(d - u)*10)**2) - 0.01*x[-1]  
        FC = cvx.sum(cvx.pos(u - umax)**2)
       
        # objective function
        obj = cvx.Minimize(FC+WS)    

        # initial condition
        if i==0:
            ic = df_Q.storage.values[0]/1000
        else:
            ic = x_save[i] # from previous optimization

        # constraints (define separately, then concatenate the lists)
        c_mass_balance = [x[1:] == x[:-1] - u + Q[i:i+h]] # state transition
        c_release = [u >= 0] # release lower/upper bounds 
        c_storage = [x >= 0, x <= K] # storage lower/upper bounds
        c_init_final = [x[0] == ic, x[h] >= 0]
        constraints = c_mass_balance + c_release + c_storage + c_init_final

        prob = cvx.Problem(obj, constraints)
    
        if (solver == 'ECOS'):
            prob.solve(solver='ECOS')
        else:
            prob.solve()  
        
        # if solver is wrong, print details
        if (detail == 'y') & (prob.status!='optimal'):
            print(i)
            print('Status: %s' % prob.status)
            print('Obj Fun: %f' % obj.value)  

        # save in big array
        collective_obj += (obj.value + x.value[h])/h
        x_save[i+1] = x.value[1]
        u_save[i] = u.value[0]
        shortage_save[i] = max(5-u.value[0],0)
        spill_save[i] = max(u.value[0]-5,0)
        
    # add these results into the original dataframe
    x_save[0] = df_Q.storage.values[0]/1000
    end_storage = x_save[-1]
    df_Q = get_year(df_folsom, year1, 0, year2)
    df_Q.inflow = df_Q.inflow.values.astype(int)*unit_conv
    df_Q['x'] = pd.Series(x_save[:-1], index=df_Q.index)
    df_Q['u'] = pd.Series(u_save, index=df_Q.index)
    df_Q['shortage'] = pd.Series(shortage_save, index=df_Q.index)
    df_Q['spill'] = pd.Series(spill_save, index=df_Q.index)
        
    return collective_obj, end_storage, df_Q

In [7]:
def get_forecast(df_folsom, year1, h, deviation, year2=None):
    # this function creates synthetic forecast of single or multi-year 
    # based on historical flow and a specified loss of accuracy. 
    # A random normal error is introduced with a 95 percent chance that 
    # it's within the specified deviation range.  
    
    # obtain actual historical inflow as the perfect forcast
    df_Q = get_year(df_folsom, year1, h, year2)
    unit_conv = 60*60*24/43560/1000
    Q = df_Q.inflow.values.astype(int)*unit_conv
    
    # set parameter
    T = len(Q)
    Q_f = np.zeros(T)
 
    # make sure all forecast inflow is above zero 
    i = 0
    while i < T: 
        error = np.random.normal(0, deviation*Q[i]/2, 1)
        if Q[i]+error>0:
            Q_f[i]=Q[i]+error
            i += 1      
            
    # save inflow and forecast    
    df_Q.inflow = Q
    df_Q['forecast'] = Q_f
    return df_Q  

In [8]:
def folsom_forecast(df_folsom, year1, h, umax, deviation, year2=None, detail='n', solver=''):
    # run daily optimization over horizon h using forecast inflow
    # hence the forecast model  

    
    # set up parameters
    df_Q = get_forecast(df_folsom, year1, h, deviation, year2)
    F = df_Q.forecast.copy().values
    Q = df_Q.inflow.values
    T = len(Q)
    K = 975 # reservoir capacity
    d = 5*np.ones(h) # target demand (TAF/daily)

    # need some empty arrays to fill in
    x_save = np.zeros(T-h+1)
    u_save = np.zeros(T-h)
    shortage_save = np.zeros(T-h)
    spill_save = np.zeros(T-h)
    collective_obj = 0

    # loop daily for solution
    for i in range(0,T-h):
        
        # correct the value of inflow today
        F[i]=Q[i]
        
        # set up parameters
        x = cvx.Variable(h+1)
        u = cvx.Variable(h)
        WS = cvx.sum((cvx.pos(d - u)*10)**2) - 0.01*x[-1]  
        FC = cvx.sum(cvx.pos(u - umax)**2)
       
        # objective function
        obj = cvx.Minimize(FC+WS)    

        # initial condition
        if i==0:
            ic = df_Q.storage.values[0]/1000
        else:
            ic = x_save[i] # from previous optimization

        # constraints (define separately, then concatenate the lists)
        c_mass_balance = [x[1:] == x[:-1] - u + F[i:i+h]] # state transition
        c_release = [u >= 0] # release lower/upper bounds 
        c_storage = [x >= 0, x <= K] # storage lower/upper bounds
        c_init_final = [x[0] == ic, x[h] >= 0]
        constraints = c_mass_balance + c_release + c_storage + c_init_final

        prob = cvx.Problem(obj, constraints)
    
        if (solver == 'ECOS'):
            prob.solve(solver='ECOS')
        else:
            prob.solve()  
        
        # if solver is wrong, print details
        if (detail == 'y') & (prob.status!='optimal'):
            print(i)
            print('Status: %s' % prob.status)
            print('Obj Fun: %f' % obj.value)  

        # save in big array
        collective_obj += (obj.value + x.value[h])/h
        x_save[i+1] = x.value[1]
        u_save[i] = u.value[0]
        shortage_save[i] = max(5-u.value[0],0)
        spill_save[i] = max(u.value[0]-5,0)
        
    # add these results into the original dataframe
    x_save[0] = df_Q.storage.values[0]/1000
    end_storage = x_save[-1]
    df_Q = df_Q[:-h]    
    df_Q['x'] = pd.Series(x_save[:-1], index=df_Q.index)
    df_Q['u'] = pd.Series(u_save, index=df_Q.index)
    df_Q['shortage'] = pd.Series(shortage_save, index=df_Q.index)
    df_Q['spill'] = pd.Series(spill_save, index=df_Q.index)
        
    return collective_obj, end_storage, df_Q 

In [9]:
def SLOP(df_folsom, year1, year2=None):
    # runs SLOP model on a single or multiyear input
    # returns the time series result of release in dataframe format
    
    # get data
    df_Q = get_year(df_folsom, year1, 0, year2)
    unit_conv = 60*60*24/43560/1000
    Q = df_Q.inflow.values.astype(int)*unit_conv
    
    # set up parameter
    T = len(Q)
    K = 975 # reservoir capacity
    d = 5
    
    S = np.empty(T+1)     #array of annual storage
    R = np.empty(T)       #array of annual release
    S[0] = df_Q.storage.values[0]/1000   #reservior starts with historical value 

    for t in range(T):
        if S[t] + Q[t] > K:
            R[t] = S[t] + Q[t] - K
        elif S[t] + Q[t] > d:
            R[t] = d
        else:
            R[t] = S[t] + Q[t]        
    
        S[t+1] = S[t] + Q[t] - R[t]
    
    df_Q.inflow = Q
    df_Q['x'] = S[:-1]
    df_Q['u'] = R
    df_Q['shortage'] = (d-R).clip(0,5)
    df_Q['spill'] = (R-d).clip(0,500)
    return df_Q